Lo primero que hacemos en el proceso de ETL de este proyecto es cargar el csv, lo cargaremos con polars usando un read_csv y lo cargaremos con el codigo ISO.


In [139]:
import polars as pl

df = pl.read_csv("data.csv", encoding="ISO-8859-1", infer_schema_length=10000)
df


InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
str,str,str,i64,str,f64,i64,str
"""536365""","""85123A""","""WHITE HANGING HEART T-LIGHT HO…",6,"""12/1/2010 8:26""",2.55,17850,"""United Kingdom"""
"""536365""","""71053""","""WHITE METAL LANTERN""",6,"""12/1/2010 8:26""",3.39,17850,"""United Kingdom"""
"""536365""","""84406B""","""CREAM CUPID HEARTS COAT HANGER""",8,"""12/1/2010 8:26""",2.75,17850,"""United Kingdom"""
"""536365""","""84029G""","""KNITTED UNION FLAG HOT WATER B…",6,"""12/1/2010 8:26""",3.39,17850,"""United Kingdom"""
"""536365""","""84029E""","""RED WOOLLY HOTTIE WHITE HEART.""",6,"""12/1/2010 8:26""",3.39,17850,"""United Kingdom"""
…,…,…,…,…,…,…,…
"""581587""","""22613""","""PACK OF 20 SPACEBOY NAPKINS""",12,"""12/9/2011 12:50""",0.85,12680,"""France"""
"""581587""","""22899""","""CHILDREN'S APRON DOLLY GIRL """,6,"""12/9/2011 12:50""",2.1,12680,"""France"""
"""581587""","""23254""","""CHILDRENS CUTLERY DOLLY GIRL """,4,"""12/9/2011 12:50""",4.15,12680,"""France"""



Ahora sobre el dataset vamos a analizar si hay datos que limpiar:

Primero comprobaremos si el dataset contiene valores nulos.

In [140]:
df.null_count()


InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
u32,u32,u32,u32,u32,u32,u32,u32
0,0,1454,0,0,0,135080,0


Como vemos hay muchos nulls, dado que el objetivo es agrupar por tipo de clientes el custumerID es esencial y no podemos tabajar sin el. y la descripcion seguramente la ignoremos, por que lo mas util a la hora de agrupar por cliente es la cantidad comprada, el valor total de gasto  y pais.

In [141]:
df = df.filter(df["Description"].is_not_null())
df = df.filter(df["CustomerID"].is_not_null())
df

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
str,str,str,i64,str,f64,i64,str
"""536365""","""85123A""","""WHITE HANGING HEART T-LIGHT HO…",6,"""12/1/2010 8:26""",2.55,17850,"""United Kingdom"""
"""536365""","""71053""","""WHITE METAL LANTERN""",6,"""12/1/2010 8:26""",3.39,17850,"""United Kingdom"""
"""536365""","""84406B""","""CREAM CUPID HEARTS COAT HANGER""",8,"""12/1/2010 8:26""",2.75,17850,"""United Kingdom"""
"""536365""","""84029G""","""KNITTED UNION FLAG HOT WATER B…",6,"""12/1/2010 8:26""",3.39,17850,"""United Kingdom"""
"""536365""","""84029E""","""RED WOOLLY HOTTIE WHITE HEART.""",6,"""12/1/2010 8:26""",3.39,17850,"""United Kingdom"""
…,…,…,…,…,…,…,…
"""581587""","""22613""","""PACK OF 20 SPACEBOY NAPKINS""",12,"""12/9/2011 12:50""",0.85,12680,"""France"""
"""581587""","""22899""","""CHILDREN'S APRON DOLLY GIRL """,6,"""12/9/2011 12:50""",2.1,12680,"""France"""
"""581587""","""23254""","""CHILDRENS CUTLERY DOLLY GIRL """,4,"""12/9/2011 12:50""",4.15,12680,"""France"""


Habiendonos desecho de todos los datos con valores nulos seguiremos con los valores duplicados de todas las columnas, que tambien es importante por si a la hora de insertar los datos se cometio el error de añadir la misma compra dos veces, con lo cual no nos seria util para nuestra posterior predicción.

In [142]:

f"Filas duplicadas: {df.height - df.unique().height}"

'Filas duplicadas: 5225'

En nuestro caso elimineramos estas filas duplicadas y, como mencionamos anteriormente, es muy probable que se trate de un error de la aplicacion al insertar datos y nos perjudique a la hora de hacer las predicciones.

In [143]:

df = df.unique()
df

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
str,str,str,i64,str,f64,i64,str
"""559672""","""85071B""","""RED CHARLIE+LOLA PERSONAL DOOR…",18,"""7/11/2011 14:04""",0.39,14796,"""United Kingdom"""
"""562129""","""22430""","""ENAMEL WATERING CAN CREAM""",4,"""8/3/2011 9:09""",4.95,13186,"""United Kingdom"""
"""568060""","""84945""","""MULTI COLOUR SILVER T-LIGHT HO…",12,"""9/23/2011 13:11""",0.85,14546,"""United Kingdom"""
"""575837""","""23483""","""HANGING BUTTERFLY T-LIGHT HOL…",18,"""11/11/2011 11:37""",1.25,12748,"""United Kingdom"""
"""566408""","""23415""","""HOME SWEET HOME HOOK""",5,"""9/12/2011 13:43""",1.65,16984,"""United Kingdom"""
…,…,…,…,…,…,…,…
"""570637""","""85099B""","""JUMBO BAG RED RETROSPOT""",10,"""10/11/2011 12:34""",2.08,12530,"""Germany"""
"""552883""","""21672""","""WHITE SPOT RED CERAMIC DRAWER …",96,"""5/12/2011 10:13""",1.06,14646,"""Netherlands"""
"""555405""","""23147""","""SINGLE ANTIQUE ROSE HOOK IVORY""",12,"""6/2/2011 19:17""",1.45,17531,"""United Kingdom"""


Ahora iremos con los valores negativos tanto en la columna de Quantity como en el UnitPrice, ya que serian los unicos que podrian tener unos valores negativos.

In [144]:
df.filter((pl.col("Quantity") < 0) | (pl.col("UnitPrice") < 0))


InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
str,str,str,i64,str,f64,i64,str
"""C558361""","""21429""","""RED GINGHAM ROSE JEWELLERY BOX""",-2,"""6/28/2011 15:13""",1.95,15128,"""United Kingdom"""
"""C550962""","""21754""","""HOME BUILDING BLOCK WORD""",-1,"""4/21/2011 16:42""",5.95,14834,"""United Kingdom"""
"""C569655""","""22477""","""WATERING CAN GARDEN MARKER""",-1,"""10/5/2011 12:58""",1.25,14534,"""United Kingdom"""
"""C566784""","""85048""","""15CM CHRISTMAS GLASS BALL 20 L…",-7,"""9/15/2011 10:27""",6.95,14915,"""United Kingdom"""
"""C554849""","""22055""","""MINI CAKE STAND HANGING STRAW…",-1,"""5/26/2011 19:58""",1.45,17865,"""United Kingdom"""
…,…,…,…,…,…,…,…
"""C551015""","""23176""","""ABC TREASURE BOOK BOX """,-4,"""4/26/2011 11:06""",2.25,12753,"""Japan"""
"""C571316""","""23483""","""HANGING BUTTERFLY T-LIGHT HOL…",-3,"""10/17/2011 10:47""",1.25,15091,"""United Kingdom"""
"""C550181""","""22381""","""TOY TIDY PINK POLKADOT""",-10,"""4/14/2011 17:43""",2.1,13069,"""United Kingdom"""


En este caso vemos que hay valores negativos en estas dos columnas, se podrian interpretar como devoluciones de los productos, pero en este caso no los vamos a usar de esta forma porque no tendria sentido contar con estos valores si solo queremos calcular las ventas y no devoluciones. Asi que los tendremos que eliminar.

In [145]:
df = df.filter(~((pl.col("Quantity") < 0) | (pl.col("UnitPrice") < 0)))
df


InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
str,str,str,i64,str,f64,i64,str
"""559672""","""85071B""","""RED CHARLIE+LOLA PERSONAL DOOR…",18,"""7/11/2011 14:04""",0.39,14796,"""United Kingdom"""
"""562129""","""22430""","""ENAMEL WATERING CAN CREAM""",4,"""8/3/2011 9:09""",4.95,13186,"""United Kingdom"""
"""568060""","""84945""","""MULTI COLOUR SILVER T-LIGHT HO…",12,"""9/23/2011 13:11""",0.85,14546,"""United Kingdom"""
"""575837""","""23483""","""HANGING BUTTERFLY T-LIGHT HOL…",18,"""11/11/2011 11:37""",1.25,12748,"""United Kingdom"""
"""566408""","""23415""","""HOME SWEET HOME HOOK""",5,"""9/12/2011 13:43""",1.65,16984,"""United Kingdom"""
…,…,…,…,…,…,…,…
"""570637""","""85099B""","""JUMBO BAG RED RETROSPOT""",10,"""10/11/2011 12:34""",2.08,12530,"""Germany"""
"""552883""","""21672""","""WHITE SPOT RED CERAMIC DRAWER …",96,"""5/12/2011 10:13""",1.06,14646,"""Netherlands"""
"""555405""","""23147""","""SINGLE ANTIQUE ROSE HOOK IVORY""",12,"""6/2/2011 19:17""",1.45,17531,"""United Kingdom"""


Es el turno de la fecha de facturacion, vamos a transformar la columna a tipo date

In [146]:
df = df.with_columns(pl.col("InvoiceDate").str.strptime(pl.Date, "%m/%d/%Y %H:%M"))
df


InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
str,str,str,i64,date,f64,i64,str
"""559672""","""85071B""","""RED CHARLIE+LOLA PERSONAL DOOR…",18,2011-07-11,0.39,14796,"""United Kingdom"""
"""562129""","""22430""","""ENAMEL WATERING CAN CREAM""",4,2011-08-03,4.95,13186,"""United Kingdom"""
"""568060""","""84945""","""MULTI COLOUR SILVER T-LIGHT HO…",12,2011-09-23,0.85,14546,"""United Kingdom"""
"""575837""","""23483""","""HANGING BUTTERFLY T-LIGHT HOL…",18,2011-11-11,1.25,12748,"""United Kingdom"""
"""566408""","""23415""","""HOME SWEET HOME HOOK""",5,2011-09-12,1.65,16984,"""United Kingdom"""
…,…,…,…,…,…,…,…
"""570637""","""85099B""","""JUMBO BAG RED RETROSPOT""",10,2011-10-11,2.08,12530,"""Germany"""
"""552883""","""21672""","""WHITE SPOT RED CERAMIC DRAWER …",96,2011-05-12,1.06,14646,"""Netherlands"""
"""555405""","""23147""","""SINGLE ANTIQUE ROSE HOOK IVORY""",12,2011-06-02,1.45,17531,"""United Kingdom"""


Lo siguiente sera buscar algun outliers, para evitar la distorsion de los patrones, la no normalizacion de los datos y que no afecten a la precisión del modelo.

In [147]:
df.describe()


statistic,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
str,str,str,str,f64,str,f64,f64,str
"""count""","""392732""","""392732""","""392732""",392732.0,"""392732""",392732.0,392732.0,"""392732"""
"""null_count""","""0""","""0""","""0""",0.0,"""0""",0.0,0.0,"""0"""
"""mean""",null,null,null,13.153718,"""2011-07-10 06:01:42.518000""",3.125596,15287.734822,null
"""std""",null,null,null,181.58842,null,22.240725,1713.567773,null
"""min""","""536365""","""10002""",""" 4 PURPLE FLOCK DINNER CANDLES""",1.0,"""2010-12-01""",0.0,12346.0,"""Australia"""
"""25%""",null,null,null,2.0,"""2011-04-07""",1.25,13955.0,null
"""50%""",null,null,null,6.0,"""2011-07-31""",1.95,15150.0,null
"""75%""",null,null,null,12.0,"""2011-10-20""",3.75,16791.0,null
"""max""","""581587""","""POST""","""ZINC WIRE SWEETHEART LETTER TR…",80995.0,"""2011-12-09""",8142.75,18287.0,"""Unspecified"""


no vamos a liminar las compras que tengan precio de 0, porque entendemos que son regalos de la tienda y ademas nos puede venir bien para agrupar clientes que solo compren cuando hay cosas gratuitas

Siguiendo con las cosas un poco raras que habiamos detectado, en la columna Country habiamos visto un "unspecified"

In [148]:
df.filter(pl.col("Country") == "Unspecified" )


InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
str,str,str,i64,date,f64,i64,str
"""559929""","""22960""","""JAM MAKING SET WITH JARS""",6,2011-07-14,4.25,14265,"""Unspecified"""
"""552695""","""22413""","""METAL SIGN TAKE IT OR LEAVE IT…",6,2011-05-10,2.95,16320,"""Unspecified"""
"""561658""","""22604""","""SET OF 4 NAPKIN CHARMS CUTLERY""",2,2011-07-28,2.55,12743,"""Unspecified"""
"""561661""","""22289""","""HANGING METAL CHICKEN DECORATI…",2,2011-07-28,1.25,12743,"""Unspecified"""
"""552695""","""22960""","""JAM MAKING SET WITH JARS""",12,2011-05-10,3.75,16320,"""Unspecified"""
…,…,…,…,…,…,…,…
"""561658""","""21002""","""ROSE DU SUD DRAWSTRING BAG""",2,2011-07-28,4.25,12743,"""Unspecified"""
"""564051""","""23046""","""PAPER LANTERN 9 POINT DELUXE S…",2,2011-08-22,6.65,14265,"""Unspecified"""
"""559929""","""21628""","""TRIANGULAR POUFFE VINTAGE """,2,2011-07-14,14.95,14265,"""Unspecified"""


Como en nuestro caso nos parece util y queremos llegar a agrupar las ventas por paises debemos eliminar las filas que contengan "Unspecified" en la columna Country.

In [149]:
df = df.filter(pl.col("Country") != "Unspecified" )
df

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
str,str,str,i64,date,f64,i64,str
"""559672""","""85071B""","""RED CHARLIE+LOLA PERSONAL DOOR…",18,2011-07-11,0.39,14796,"""United Kingdom"""
"""562129""","""22430""","""ENAMEL WATERING CAN CREAM""",4,2011-08-03,4.95,13186,"""United Kingdom"""
"""568060""","""84945""","""MULTI COLOUR SILVER T-LIGHT HO…",12,2011-09-23,0.85,14546,"""United Kingdom"""
"""575837""","""23483""","""HANGING BUTTERFLY T-LIGHT HOL…",18,2011-11-11,1.25,12748,"""United Kingdom"""
"""566408""","""23415""","""HOME SWEET HOME HOOK""",5,2011-09-12,1.65,16984,"""United Kingdom"""
…,…,…,…,…,…,…,…
"""570637""","""85099B""","""JUMBO BAG RED RETROSPOT""",10,2011-10-11,2.08,12530,"""Germany"""
"""552883""","""21672""","""WHITE SPOT RED CERAMIC DRAWER …",96,2011-05-12,1.06,14646,"""Netherlands"""
"""555405""","""23147""","""SINGLE ANTIQUE ROSE HOOK IVORY""",12,2011-06-02,1.45,17531,"""United Kingdom"""


Algo raro tambien es que habia valores maximos muy altos en la cantidad y en el precio, es decir tenemos claramente outliers que habiamos detectado antes. Para solucionar esto usaremos la formula del IQR para ddefinir un limite superior e inderior en los valores.
los modelos de agrupacion son muy sensibles a los outliers

In [150]:
def calc_iqr_from_column_name(name:str)-> None:
    Q1 = df[name].quantile(0.25)
    Q3 = df[name].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    print(f"\nIQR for {name}")
    print(Q1, Q3, IQR, lower_bound, upper_bound)

calc_iqr_from_column_name("Quantity")
calc_iqr_from_column_name("UnitPrice")


IQR for Quantity
2.0 12.0 10.0 -13.0 27.0

IQR for UnitPrice
1.25 3.75 2.5 -2.5 7.5


In [151]:
upper_bound_limit_condition = (pl.col("Quantity") > 27) | (pl.col("UnitPrice") > 7.5)

df.filter(upper_bound_limit_condition)

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
str,str,str,i64,date,f64,i64,str
"""570669""","""23541""","""WALL ART CLASSIC PUDDINGS """,2,2011-10-11,8.25,15955,"""United Kingdom"""
"""553671""","""22501""","""PICNIC BASKET WICKER LARGE""",1,2011-05-18,9.95,13659,"""United Kingdom"""
"""537126""","""22635""","""CHILDS BREAKFAST SET DOLLY GIR…",2,2010-12-05,9.95,18118,"""United Kingdom"""
"""539579""","""22171""","""3 HOOK PHOTO SHELF ANTIQUE WHI…",2,2010-12-20,8.5,16713,"""United Kingdom"""
"""574882""","""23558""","""LANDMARK FRAME LONDON BRIDGE """,1,2011-11-07,12.5,13599,"""United Kingdom"""
…,…,…,…,…,…,…,…
"""561320""","""82484""","""WOOD BLACK BOARD ANT WHITE FIN…",3,2011-07-26,7.95,16942,"""United Kingdom"""
"""561095""","""22423""","""REGENCY CAKESTAND 3 TIER""",16,2011-07-25,10.95,17040,"""United Kingdom"""
"""542098""","""22526""","""WHEELBARROW FOR CHILDREN """,2,2011-01-25,12.75,16531,"""United Kingdom"""


LLevando a cabo esta accion estariamos eliminando bastantes datos, pero la parte positiva es que estariamos normalizando lo maximo posible el flujo de los datos, entonces procederemos a eliminarlos.

In [152]:
df = df.filter(~((pl.col("Quantity") > 27) | (pl.col("UnitPrice") > 7.5)))
df

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
str,str,str,i64,date,f64,i64,str
"""559672""","""85071B""","""RED CHARLIE+LOLA PERSONAL DOOR…",18,2011-07-11,0.39,14796,"""United Kingdom"""
"""562129""","""22430""","""ENAMEL WATERING CAN CREAM""",4,2011-08-03,4.95,13186,"""United Kingdom"""
"""568060""","""84945""","""MULTI COLOUR SILVER T-LIGHT HO…",12,2011-09-23,0.85,14546,"""United Kingdom"""
"""575837""","""23483""","""HANGING BUTTERFLY T-LIGHT HOL…",18,2011-11-11,1.25,12748,"""United Kingdom"""
"""566408""","""23415""","""HOME SWEET HOME HOOK""",5,2011-09-12,1.65,16984,"""United Kingdom"""
…,…,…,…,…,…,…,…
"""563555""","""22201""","""FRYING PAN BLUE POLKADOT""",1,2011-08-17,4.25,16755,"""United Kingdom"""
"""570637""","""85099B""","""JUMBO BAG RED RETROSPOT""",10,2011-10-11,2.08,12530,"""Germany"""
"""555405""","""23147""","""SINGLE ANTIQUE ROSE HOOK IVORY""",12,2011-06-02,1.45,17531,"""United Kingdom"""


In [153]:
df.describe()

statistic,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
str,str,str,str,f64,str,f64,f64,str
"""count""","""333055""","""333055""","""333055""",333055.0,"""333055""",333055.0,333055.0,"""333055"""
"""null_count""","""0""","""0""","""0""",0.0,"""0""",0.0,0.0,"""0"""
"""mean""",null,null,null,7.550321,"""2011-07-12 00:32:44.821000""",2.194007,15319.817204,null
"""std""",null,null,null,6.775819,null,1.546041,1710.841623,null
"""min""","""536365""","""10002""",""" 4 PURPLE FLOCK DINNER CANDLES""",1.0,"""2010-12-01""",0.0,12347.0,"""Australia"""
"""25%""",null,null,null,2.0,"""2011-04-08""",1.25,13988.0,null
"""50%""",null,null,null,6.0,"""2011-08-03""",1.65,15235.0,null
"""75%""",null,null,null,12.0,"""2011-10-23""",2.95,16814.0,null
"""max""","""581587""","""POST""","""ZINC WIRE SWEETHEART LETTER TR…",27.0,"""2011-12-09""",7.5,18287.0,"""United Kingdom"""


Viendo que el objetivo es agrupar en diferentes tipos de clientes eliminaremos las columnas que no son relevantes

CustomerID,InvoiceDate,Quantity, UnitPrice,InvoiceNo, Country: serian las columnas mas interesantes para nuestro modelo de clustering, teniendo en cuenta que country seria opcional y solo seria util si queremos segmentar internacionalmente

In [154]:
df = df["CustomerID","InvoiceDate","Quantity", "UnitPrice","Country","InvoiceNo"]
df

CustomerID,InvoiceDate,Quantity,UnitPrice,Country,InvoiceNo
i64,date,i64,f64,str,str
14796,2011-07-11,18,0.39,"""United Kingdom""","""559672"""
13186,2011-08-03,4,4.95,"""United Kingdom""","""562129"""
14546,2011-09-23,12,0.85,"""United Kingdom""","""568060"""
12748,2011-11-11,18,1.25,"""United Kingdom""","""575837"""
16984,2011-09-12,5,1.65,"""United Kingdom""","""566408"""
…,…,…,…,…,…
16755,2011-08-17,1,4.25,"""United Kingdom""","""563555"""
12530,2011-10-11,10,2.08,"""Germany""","""570637"""
17531,2011-06-02,12,1.45,"""United Kingdom""","""555405"""


Ahora es el turno de las columnas extra.
Hemos pensado las siguientes:

TotalPrice ->  Para saber cuanto gastan los clientes.
Last_buy ->  Días desde la última compra.
Frequency ->  Número de compras (facturas únicas) por cliente.
Monetary -> Suma del gasto total por cliente.

Creando la variable de totalPrice

In [155]:
df = df.with_columns(
    (pl.col("Quantity") * pl.col("UnitPrice")).alias("TotalPrice")
)
df

CustomerID,InvoiceDate,Quantity,UnitPrice,Country,InvoiceNo,TotalPrice
i64,date,i64,f64,str,str,f64
14796,2011-07-11,18,0.39,"""United Kingdom""","""559672""",7.02
13186,2011-08-03,4,4.95,"""United Kingdom""","""562129""",19.8
14546,2011-09-23,12,0.85,"""United Kingdom""","""568060""",10.2
12748,2011-11-11,18,1.25,"""United Kingdom""","""575837""",22.5
16984,2011-09-12,5,1.65,"""United Kingdom""","""566408""",8.25
…,…,…,…,…,…,…
16755,2011-08-17,1,4.25,"""United Kingdom""","""563555""",4.25
12530,2011-10-11,10,2.08,"""Germany""","""570637""",20.8
17531,2011-06-02,12,1.45,"""United Kingdom""","""555405""",17.4


ahora Last_buy, frequency y monetary

Ahora agruparemos por dias para ver la suma de todas las ventas totales. De esta forma tendremos una mejor visualizacion de todas las ventas totales en cada dia que se han realizado.


In [156]:
max_date = df.select(pl.col("InvoiceDate").max()).to_series()[0]

df = df.group_by("CustomerID").agg([

    ((pl.lit(max_date) - pl.col("InvoiceDate").max()) / pl.duration(days=1)).alias("Last_buy"),
      

    pl.col("InvoiceNo").n_unique().alias("Frequency"),
    

    pl.col("TotalPrice").sum().alias("Monetary")
])
df


CustomerID,Last_buy,Frequency,Monetary
i64,f64,u32,f64
14687,106.0,1,217.02
12594,46.0,8,2743.43
15214,1.0,8,1497.14
16387,322.0,1,94.36
17962,142.0,1,82.02
…,…,…,…
16637,32.0,4,921.76
13496,10.0,2,563.63
15205,85.0,2,425.25


Ahora es el turno de otro describe con las nuevas variables

In [157]:
df.describe()

statistic,CustomerID,Last_buy,Frequency,Monetary
str,f64,f64,f64,f64
"""count""",4188.0,4188.0,4188.0,4188.0
"""null_count""",0.0,0.0,0.0,0.0
"""mean""",15291.565186,91.940306,4.017908,1026.10236
"""std""",1718.981692,99.933739,7.025996,2200.968737
"""min""",12347.0,0.0,1.0,0.0
"""25%""",13810.0,17.0,1.0,206.09
"""50%""",15286.0,50.0,2.0,465.97
"""75%""",16771.0,143.0,4.0,1128.05
"""max""",18287.0,373.0,197.0,84980.89


como vemos en monatery y en frequency tenemos claramente outliers, aplicaremos el logaritmo para no perder informacion y reducir la gran diferencia que hay, eso si tenemos que tener cuidado con los ceros

no lo aplicaremos sobre last_buy porque no tiene una destribucion tan extrema como monetary o frequency y los dias son mas uniformes lo que puede probocar que el logaritmo distorsione la interpretacion.

In [158]:
df = df.filter(df["Monetary"]>0)
df = df.with_columns(
    (pl.col("Monetary") + 1).log().alias("Monetary_log"),
    (pl.col("Frequency") + 1).log().alias("Frequency_log")
)
df

CustomerID,Last_buy,Frequency,Monetary,Monetary_log,Frequency_log
i64,f64,u32,f64,f64,f64
14687,106.0,1,217.02,5.384587,0.693147
12594,46.0,8,2743.43,7.917329,2.197225
15214,1.0,8,1497.14,7.31198,2.197225
16387,322.0,1,94.36,4.557659,0.693147
17962,142.0,1,82.02,4.419082,0.693147
…,…,…,…,…,…
16637,32.0,4,921.76,6.827369,1.609438
13496,10.0,2,563.63,6.336171,1.098612
15205,85.0,2,425.25,6.055026,1.098612


Con esto tendriamos casi listo el proceso de ETL

Faltaria hacer algunos pasos que son recomendados como el escalado dde variables ya que las metricas estan en escalas muy diferentes.

y una gran diferencia en a escala afecta al clustering
usaremos un standarscaler, tambien prodiamos usar el minmaxscaler o el Z

In [159]:
from sklearn.preprocessing import StandardScaler

X = df.select(["Last_buy", "Frequency_log", "Monetary_log"]).to_numpy()


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [160]:
df = df.with_columns([
    pl.Series("Last_buy_scaled", X_scaled[:, 0]),
    pl.Series("Frequency_scaled", X_scaled[:, 1]),
    pl.Series("Monetary_scaled", X_scaled[:, 2])
])
df

CustomerID,Last_buy,Frequency,Monetary,Monetary_log,Frequency_log,Last_buy_scaled,Frequency_scaled,Monetary_scaled
i64,f64,u32,f64,f64,f64,f64,f64,f64
14687,106.0,1,217.02,5.384587,0.693147,0.140529,-0.948408,-0.627818
12594,46.0,8,2743.43,7.917329,2.197225,-0.459905,1.325446,1.402687
15214,1.0,8,1497.14,7.31198,2.197225,-0.91023,1.325446,0.917377
16387,322.0,1,94.36,4.557659,0.693147,2.302091,-0.948408,-1.290768
17962,142.0,1,82.02,4.419082,0.693147,0.500789,-0.948408,-1.401866
…,…,…,…,…,…,…,…,…
16637,32.0,4,921.76,6.827369,1.609438,-0.600006,0.436834,0.528863
13496,10.0,2,563.63,6.336171,1.098612,-0.820165,-0.335428,0.135069
15205,85.0,2,425.25,6.055026,1.098612,-0.069623,-0.335428,-0.090326


Aqui hacemos un ultimo describe para ver como ha quedado el dataset, con todos los cambios

In [161]:
df.describe()

statistic,CustomerID,Last_buy,Frequency,Monetary,Monetary_log,Frequency_log,Last_buy_scaled,Frequency_scaled,Monetary_scaled
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",4187.0,4187.0,4187.0,4187.0,4187.0,4187.0,4187.0,4187.0,4187.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",15292.207308,91.957249,4.018629,1026.347429,6.167694,1.320487,-6.1093e-17,3.3940e-18,-1.1540e-16
"""std""",1718.684598,99.939659,7.02668,2201.17447,1.247495,0.661545,1.000119,1.000119,1.000119
"""min""",12347.0,0.0,1.0,1.9,1.064711,0.693147,-0.920238,-0.948408,-4.091073
"""25%""",13812.0,17.0,1.0,206.13,5.333347,0.693147,-0.750115,-0.948408,-0.668898
"""50%""",15286.0,50.0,2.0,465.97,6.146265,1.098612,-0.419876,-0.335428,-0.017179
"""75%""",16773.0,143.0,4.0,1130.31,7.031132,1.609438,0.510797,0.436834,0.69222
"""max""",18287.0,373.0,197.0,84980.89,11.350193,5.288267,2.81246,5.998462,4.154822


In [162]:
df.write_csv("data_cleaned(clustering).csv")




In [163]:
import pickle


with open("datos_grouped(clustering).pkl", "wb") as file:
    pickle.dump(df, file)

